In [1]:
import torch
from PIL import Image
import os
import re
from PIL import Image

from models.controlnext_vid_svd import ControlNeXtSDVModel
from models.unet_spatio_temporal_condition_controlnext import UNetSpatioTemporalConditionControlNeXtModel
from pipelines.pipeline_stable_video_diffusion import StableVideoDiffusionPipeline

from diffusers.utils import load_image, export_to_video, export_to_gif

/root/miniconda3/envs/diffusers/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_images_from_folder(folder):
    images = []
    valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"}  # Add or remove extensions as needed

    # Function to extract frame number from the filename
    def frame_number(filename):
        # First, try the pattern 'frame_x_7fps'
        new_pattern_match = re.search(r'frame_(\d+)_7fps', filename)
        if new_pattern_match:
            return int(new_pattern_match.group(1))
        # If the new pattern is not found, use the original digit extraction method
        matches = re.findall(r'\d+', filename)
        if matches:
            if matches[-1] == '0000' and len(matches) > 1:
                return int(matches[-2])  # Return the second-to-last sequence if the last is '0000'
            return int(matches[-1])  # Otherwise, return the last sequence
        return float('inf')  # Return 'inf'

    # Sorting files based on frame number
    sorted_files = sorted(os.listdir(folder), key=frame_number)

    # Load images in sorted order
    for filename in sorted_files:
        ext = os.path.splitext(filename)[1].lower()
        if ext in valid_extensions:
            img = Image.open(os.path.join(folder, filename)).convert('RGB')
            images.append(img)

    return images

In [ ]:
controlnext = ControlNeXtSDVModel.from_pretrained(
    "./outputs/event_motion_svd",
    # "./outputs/checkpoint-40000",
    subfolder="controlnext",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False,
)

unet = UNetSpatioTemporalConditionControlNeXtModel.from_pretrained(
    "./outputs/event_motion_svd",
    # "./outputs/checkpoint-40000",
    subfolder="unet",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False,
)

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt-1-1",
    unet=unet,
    controlnext=controlnext,
    low_cpu_mem_usage=False,
    torch_dtype=torch.float16, variant="fp16", local_files_only=True,
    cache_dir="/root/autodl-tmp/models_cache"
)

Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.83it/s]


In [15]:
validation_image_dir = "/root/autodl-tmp/TikTok_event/val_set/video_001/0002.png"
validation_control_folder = "/root/autodl-tmp/TikTok_event/event_val_set/video_001"
start_frame = 80
num_frames = 14
device = "cuda"
wdtype=torch.float16

In [ ]:
validation_image = Image.open(validation_image_dir).convert('RGB')
validation_control_images = load_images_from_folder(validation_control_folder)
validation_control_images = validation_control_images[start_frame:start_frame+num_frames]

# validation_image = validation_image.resize((576, 1024))
validation_image = validation_image.resize((1024, 640))

generator = torch.manual_seed(2024)

pipe = pipe.to(device)

with torch.inference_mode():
    frames = pipe(
        validation_image,
        validation_control_images,
        control_scale=0.3,
        num_frames=num_frames,
        width=128,
        height=256,
        # width=576,
        # height=1024,
        # width=1024,
        # height=640,
        decode_chunk_size=4,
        generator=generator,
        motion_bucket_id=127,
        fps=7, 
        num_inference_steps=25,
        noise_aug_strength=0.02
        ).frames[0]
export_to_video(frames, "generated.mp4", fps=7)
# 将frames保存到指定文件夹
# for i, frame in enumerate(frames):
#     frame.save(f"frame_{i}.png")

100%|██████████| 25/25 [00:03<00:00,  6.78it/s]
/root/miniconda3/envs/diffusers/lib/python3.8/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(


'generated.mp4'

: 